In [43]:
import pandas as pd
import dashscope


In [44]:

# 读取测试文件
def read_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines()[1:]:  # 跳过表头
            parts = line.strip().split("\t")
            target, text = parts[1], parts[2]
            data.append((target, text))
    return pd.DataFrame(data, columns=["TARGET", "TEXT"])

# 生成 prompt
def generate_prompt(row):
    return f"以下是一条用户针对【{row['TARGET']}】的评论。请判断这条评论的立场是“反对”、“中立”还是“支持”：评论：【{row['TEXT']}】你的回答只需写“反对”、“中立”或“支持”。"



In [45]:
# 选择模型
choice_list = ['qwen2-7b-instruct','qwen2-72b-instruct','llama3-8b-instruct','chatglm3-6b','llama3-70b-instruct']
llm_model = choice_list[0]

# 文件路径
test_file_path = 'weibo-test.txt'
output_file_path = f'{llm_model}-weibo-test-predictions.txt'

# 加载数据
test_data = read_data(test_file_path)


In [46]:

# 生成 prompts
test_data["PROMPT"] = test_data.apply(generate_prompt, axis=1)
# test_data = test_data.head(2)
# 持久化
test_data.to_pickle('test_data.pkl')

In [40]:
import time
# 调用 DashScope 进行批量预测
def dashscope_predict(prompts):
    results = []
    
    with open('test.txt', 'a+', encoding='utf-8') as writers: # 打开文件
        for prompt in prompts:
            messages = [
                {'role': 'system', 'content': '你是一名立场检测专家。'},
                {'role': 'user', 'content': prompt}
                ]
            time.sleep(2)
            try:
                response = dashscope.Generation.call(
                    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
                    api_key='sk-b8717756fc0f4b7290f3aa7b931c448e',
                    model=llm_model, # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                    messages=messages,
                    result_format='message',
                    max_tokens=10
                    )
                results.append(response.get("output").get("choices")[0].get("message").get("content").strip())
                
                writers.write(str(len(results)) + '\n') # 将内容追加到到文件尾部
                writers.write(results + '\n') # 将内容追加到到文件尾部
            except Exception as e:
                results.append("ERROR")  # 记录出错的情况
                print(results)
                print(response)

                print(f"Error processing prompt: {prompt}\n{e}")
                
    return results


def dashscope_predict_one(prompt):    
    messages = [
        {'role': 'system', 'content': '你是一名立场检测专家。'},
        {'role': 'user', 'content': prompt}
        ]
    try:
        response = dashscope.Generation.call(
            # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
            api_key='sk-b8717756fc0f4b7290f3aa7b931c448e',
            model=llm_model, # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
            messages=messages,
            result_format='message',
            max_tokens=10
            )
        result = response.get("output").get("choices")[0].get("message").get("content").strip()
    except Exception as e:
        result = "ERROR"
        print(response)
        print(f"Error processing prompt: {response}\n{e}\n{prompt}")
                
    return result

In [ ]:
# 调用 DashScope API 进行预测
results = []

test_data["PREDICTION"] = dashscope_predict(test_data["PROMPT"].tolist())



In [ ]:
# 保存结果
test_data.to_csv(output_file_path, sep="\t", index=False, encoding='utf-8')
print(f"预测完成，结果保存至 {output_file_path}")